# Using the ``fancytypes.PreSerializable`` class #

## A NOTE BEFORE STARTING ##

Since the ``fancytypes`` git repository tracks this notebook under its original
basename ``using_PreSerializable_class.ipynb``, we recommend that you copy the
original notebook and rename it to any other basename that is not one of the
original basenames that appear in the ``<root>/examples`` directory before
executing any of the notebook cells below, where ``<root>`` is the root of the
``fancytypes`` repository. This way you can explore the notebook by executing
and modifying cells without changing the original notebook, which is being
tracked by git.

## Table of contents ##

- [Import necessary modules](#Import-necessary-modules)
- [Introduction](#Introduction)
- [Defining the ``SliceShuffler`` class](#Defining-the-SliceShuffler-class)
- [Using the ``SliceShuffler`` class](#Using-the-SliceShuffler-class)

## Import necessary modules ##

In [1]:
# For performing deep copies.
import copy



# For general array handling and constructing random number generators.
import numpy as np

# For validating and converting objects.
import czekitout.check
import czekitout.convert



# The library that is the subject of this demonstration.
import fancytypes

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


## Introduction ##

In this notebook, we use the ``fancytypes.PreSerializable`` class to define a
class of "slice shufflers", which we define as objects that can shuffle/re-order
the elements in a slice of a given array. This is a somewhat contrived example
use of the ``fancytypes.PreSerializable`` class, however it is simple and
complete. We also test the exception-raising features of the
``fancytypes.PreSerializable`` class. You can find the documentation for the 
``fancytypes.PreSerializable`` class
[here](https://mrfitzpa.github.io/fancytypes/_autosummary/fancytypes.PreSerializable.html). 
It is recommended that you consult the documentation of this class as you 
explore the notebook.

This notebook also demonstrates how one can use the helper functions 
``fancytypes.return_validation_and_conversion_funcs``, 
``fancytypes.return_pre_serialization_funcs``, and 
``fancytypes.return_de_pre_serialization_funcs``. We use these functions to
define our class of slice shufflers. You can find the documentation for these
helper functions 
[here](https://mrfitzpa.github.io/fancytypes/_autosummary/fancytypes.html).

**Users should make sure to navigate the documentation for the version of
fancytypes that they are currently using.**

In order to execute the cells in this notebook as intended, a set of Python
libraries need to be installed in the Python environment within which the cells
of the notebook are to be executed. For this particular notebook, users need to
install:

    fancytypes
    jupyter

Users can install these libraries either via `pip`:

    pip install fancytypes[examples]

or `conda`:

    conda install -y fancytypes jupyter -c conda-forge

## Defining the ``SliceShuffler`` class ##

We define the ``SliceShuffler`` class as a subclass of the
``fancytypes.PreSerializable`` class.

In [2]:
# Define the validation and conversion functions.
def _check_and_convert_slice_obj(params):
    obj_name = "slice_obj"
    kwargs = {"obj": params[obj_name],
              "obj_name": obj_name,
              "accepted_types": (slice,)}
    czekitout.check.if_instance_of_any_accepted_types(**kwargs)
    slice_obj = copy.deepcopy(params[obj_name])

    return slice_obj

def _check_and_convert_seed(params):
    obj_name = "seed"
    kwargs = {"obj": params[obj_name], "obj_name": obj_name}
    seed = czekitout.convert.to_nonnegative_int(**kwargs)

    return seed



# Define the pre-serialization functions.
def _pre_serialize_slice_obj(slice_obj):
    serializable_rep = {"start": slice_obj.start, 
                        "stop": slice_obj.stop, 
                        "step": slice_obj.step}
    
    return serializable_rep

def _pre_serialize_seed(seed):
    serializable_rep = seed
    
    return serializable_rep



# Define the de-pre-serialization functions.
def _de_pre_serialize_slice_obj(serializable_rep):
    slice_obj = slice(serializable_rep["start"], 
                      serializable_rep["stop"], 
                      serializable_rep["step"])
    
    return slice_obj

def _de_pre_serialize_seed(serializable_rep):
    seed = serializable_rep
    
    return seed



# Define the ``SliceShuffler`` class.
class SliceShuffler(fancytypes.PreSerializable):
    ctor_param_names = ("slice_obj", "seed")
    kwargs = {"namespace_as_dict": globals(),
              "ctor_param_names": ctor_param_names}

    _validation_and_conversion_funcs_ = \
        fancytypes.return_validation_and_conversion_funcs(**kwargs)
    _pre_serialization_funcs_ = \
        fancytypes.return_pre_serialization_funcs(**kwargs)
    _de_pre_serialization_funcs_ = \
        fancytypes.return_de_pre_serialization_funcs(**kwargs)

    del ctor_param_names, kwargs
    
    def __init__(self, slice_obj, seed, skip_validation_and_conversion=False):
        ctor_params = {key: val
                       for key, val in locals().items()
                       if (key not in ("self", "__class__"))}

        # Set ``skip_cls_tests`` to ``True`` only if you are sure that the
        # class that you have defined was done so properly, i.e. without errors.
        skip_cls_tests = True

        kwargs = ctor_params
        kwargs["skip_cls_tests"] = skip_cls_tests
        fancytypes.PreSerializable.__init__(self, **kwargs)

        seed = self.core_attrs["seed"]
        self._random_generator = np.random.default_rng(seed)

        return None

    @classmethod
    def get_validation_and_conversion_funcs(cls):
        validation_and_conversion_funcs = \
            cls._validation_and_conversion_funcs_.copy()

        return validation_and_conversion_funcs

    @classmethod
    def get_pre_serialization_funcs(cls):
        pre_serialization_funcs = \
            cls._pre_serialization_funcs_.copy()

        return pre_serialization_funcs

    @classmethod
    def get_de_pre_serialization_funcs(cls):
        de_pre_serialization_funcs = \
            cls._de_pre_serialization_funcs_.copy()

        return de_pre_serialization_funcs

    def shuffle(self, array):
        try:
            array = np.array(array)
        except:
            err_msg = ("The object ``array`` must be array-like.")
            raise TypeError(err_msg)
            
        slice_obj = self.core_attrs["slice_obj"]
        array_slice = array[slice_obj]
        self._random_generator.shuffle(array_slice)
        array[slice_obj] = array_slice

        return array

## Using the ``SliceShuffler`` class ##

First let's construct a valid instance of the ``SliceShuffler`` class with 
validation and conversion of the parameters to be mapped to the 
"core attributes" enabled.

In [3]:
kwargs = {"slice_obj": slice(None, 6, 1), 
          "seed": 5.0, 
          "skip_validation_and_conversion": False}
slice_shuffler = SliceShuffler(**kwargs)

There are 3 ways of accessing the core attributes via the public API:

In [4]:
# Returns a deep copy.
core_attrs = slice_shuffler.core_attrs
print(core_attrs)

# Returns a deep copy.
core_attrs = slice_shuffler.get_core_attrs(deep_copy=True)
print(core_attrs)

# Returns a reference, i.e. no copy is made. 
core_attrs = slice_shuffler.get_core_attrs(deep_copy=False)
print(core_attrs)

{'slice_obj': slice(None, 6, 1), 'seed': 5}
{'slice_obj': slice(None, 6, 1), 'seed': 5}
{'slice_obj': slice(None, 6, 1), 'seed': 5}


There is 1 way of accessing the validation and conversion functions via the 
public API:

In [5]:
# Returns a deep copy.
validation_and_conversion_funcs = \
    slice_shuffler.validation_and_conversion_funcs

print(validation_and_conversion_funcs)

{'slice_obj': <function _check_and_convert_slice_obj at 0x7fd91c3ff560>, 'seed': <function _check_and_convert_seed at 0x7fd91c2267a0>}


There is 1 way of accessing the pre-serialization functions via the public API:

In [6]:
# Returns a deep copy.
pre_serialization_funcs = slice_shuffler.pre_serialization_funcs

print(pre_serialization_funcs)

{'slice_obj': <function _pre_serialize_slice_obj at 0x7fd91c226840>, 'seed': <function _pre_serialize_seed at 0x7fd91c2268e0>}


There is 1 way of accessing the de-pre-serialization functions via the public 
API:

In [7]:
# Returns a deep copy.
de_pre_serialization_funcs = slice_shuffler.de_pre_serialization_funcs

print(pre_serialization_funcs)

{'slice_obj': <function _pre_serialize_slice_obj at 0x7fd91c226840>, 'seed': <function _pre_serialize_seed at 0x7fd91c2268e0>}


Next let's construct a valid instance of the ``SliceShuffler`` class with 
validation and conversion of the parameters to be mapped to the core attributes
disabled. This option is desired primarily when the user wants to avoid 
potentially expensive copies and/or conversions of the parameters to be mapped 
to the core attributes. However, users must ensure that the construction 
parameters are valid and require no conversions.

In [8]:
kwargs = {"slice_obj": slice(None, 6, 1), 
          "seed": 5,  # Manually converted parameter to a `int` as required.
          "skip_validation_and_conversion": True}
slice_shuffler = SliceShuffler(**kwargs)

core_attrs = slice_shuffler.core_attrs
print(core_attrs)

{'slice_obj': slice(None, 6, 1), 'seed': 5}


Let's shuffle an array using the class.

In [9]:
array = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
shuffled_array = slice_shuffler.shuffle(array)
shuffled_array

array([1, 4, 2, 3, 5, 0, 6, 7, 8, 9])

Let's try constructing instances of the ``SliceShuffler`` class with invalid
construction parameters. The following two codes blocks return errors as 
expected:

In [10]:
# Note that by default ``skip_validation_and_conversion == False``.
kwargs = {"slice_obj": 3, "seed": 5.0}
slice_shuffler = SliceShuffler(**kwargs)

TypeError: The object ``slice_obj`` must be an instance of the class `slice`.

In [11]:
kwargs = {"slice_obj": slice(None, 6, 1), "seed": "foo"}
slice_shuffler = SliceShuffler(slice_obj=slice(None, 6, 1), seed="foo")

TypeError: The object ``seed`` must be an integer.

Construct a valid instance of the ``SliceShuffler`` class and print the core attributes.

Let's pre-serialize instance. You could then serialize the serializable 
representation using the ``json`` library.

In [12]:
serializable_rep = slice_shuffler.pre_serialize()
print(serializable_rep)

{'slice_obj': {'start': None, 'stop': 6, 'step': 1}, 'seed': 5}


Alternatively, you could serialize the instance of the class by using the
``dumps`` method:

In [13]:
serialized_rep = slice_shuffler.dumps()
print(serialized_rep)

{"slice_obj": {"start": null, "stop": 6, "step": 1}, "seed": 5}


You can also serialize the instance and save the result to a JSON file in one go
using the ``dump`` method:

In [14]:
filename = "slice_shuffler.json"
slice_shuffler.dump(filename, overwrite=True)

Trying to save a serialized representation to a pre-existing that with
``overwrite==False`` will raise an exception:

In [15]:
filename = "slice_shuffler.json"
slice_shuffler.dump(filename, overwrite=False)

OSError: Cannot save the serialized representation to a file at the path ``'slice_shuffler.json'`` because a file already exists there and the object ``overwrite`` was set to ``False``, which prohibits overwriting the original file.

Let's reconstruct the instance of ``SliceShuffler`` from the serialized
representation.

In [16]:
slice_shuffler = SliceShuffler.loads(serialized_rep)
print(slice_shuffler.core_attrs)

{'slice_obj': slice(None, 6, 1), 'seed': 5}


Let's reconstruct the instance of ``SliceShuffler`` from the serialized
representation saved in the JSON file.

In [17]:
slice_shuffler = SliceShuffler.load(filename)
print(slice_shuffler.core_attrs)

{'slice_obj': slice(None, 6, 1), 'seed': 5}


Let's de-pre-serialize, i.e. construct an instance from the serializable 
representation generated above.

In [18]:
slice_shuffler = SliceShuffler.de_pre_serialize(serializable_rep)
print(slice_shuffler.core_attrs)

{'slice_obj': slice(None, 6, 1), 'seed': 5}


Note that being a direct subclass of the ``fancytypes.PreSerializable`` class,
the ``SliceShuffler`` class supports validation upon construction, and supports
pre-serialization and de-pre-serialization, but it does not support updatable
core attributes.

The ``fancytypes.Checkable`` class only supports validation upon constructing
instances.

The ``fancytypes.Updatable`` class supports updatable core attributes, and
validation upon constructing or updating instances, but it does not support
pre-serialization or de-pre-serialization.

The ``fancytypes.PreSerializableAndUpdatable`` class supports pre-serialization,
de-serialization, updatable core attributes, and enforces validation upon
constructing or updating instances.